In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from ops import *
import cv2
import os

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time

In [ ]:
path = './Font_data/'

In [ ]:
global params

params = {'path' : path,
          'batch_size' : 32,
          'output_size': 256,
          'gf_dim': 32,
          'df_dim': 32,
          'L1_lambda': 100,
          'lr': 0.0001,
          'beta1': 0.5,
          'epochs': 200}

In [ ]:
def get_file_paths(path):
    paths = sorted([os.path.join(root, file)  for root, dirs, files in os.walk(path) for file in files])
    return paths[:int(len(paths)*.8)], paths[int(len(paths)*.8):]  #trainpaths, testpaths

In [ ]:
def load_data(path):   
    img = np.expand_dims(cv2.imread(path,0),-1)
    img_ = img[:,img.shape[1]//2-128:img.shape[1]//2+128]
    if img_.shape[1]==256:
        return img_/127.5 - 1.
    else:
        return np.expand_dims(cv2.resize(img_,(256,512)),-1)/127.5 - 1.

In [ ]:
#plt.imshow(a[0][5,:,:,0])
#plt.show()

In [ ]:
global d_bn1, d_bn2, d_bn3

global g_bn_e2, g_bn_e3, g_bn_e4, g_bn_e5, g_bn_e6, g_bn_e7, g_bn_e8

global g_bn_d1, g_bn_d2, g_bn_d3, g_bn_d4, g_bn_d5, g_bn_d6 , g_bn_d7

d_bn1 = batch_norm(name='d_bn1')
d_bn2 = batch_norm(name='d_bn2')
d_bn3 = batch_norm(name='d_bn3')

g_bn_e2 = batch_norm(name='g_bn_e2')
g_bn_e3 = batch_norm(name='g_bn_e3')
g_bn_e4 = batch_norm(name='g_bn_e4')
g_bn_e5 = batch_norm(name='g_bn_e5')
g_bn_e6 = batch_norm(name='g_bn_e6')
g_bn_e7 = batch_norm(name='g_bn_e7')
g_bn_e8 = batch_norm(name='g_bn_e8')

g_bn_d1 = batch_norm(name='g_bn_d1')
g_bn_d2 = batch_norm(name='g_bn_d2')
g_bn_d3 = batch_norm(name='g_bn_d3')
g_bn_d4 = batch_norm(name='g_bn_d4')
g_bn_d5 = batch_norm(name='g_bn_d5')
g_bn_d6 = batch_norm(name='g_bn_d6')
g_bn_d7 = batch_norm(name='g_bn_d7')

In [ ]:
def generator(image, y=None):
    
    s = params['output_size']
    output_c_dim = 1
    s2, s4, s8, s16, s32, s64, s128 = int(s/2), int(s/4), int(s/8), int(s/16), int(s/32), int(s/64), int(s/128)
    gf_dim = params['gf_dim']
        
    with tf.variable_scope("generator") as scope:



        # image is (256 x 256 x input_c_dim)
        e1 = conv2d(image, gf_dim, name='g_e1_conv')
        # e1 is (128 x 128 x gf_dim)
        e2 = g_bn_e2(conv2d(lrelu(e1), gf_dim*2, name='g_e2_conv'))
        # e2 is (64 x 64 x gf_dim*2)
        e3 = g_bn_e3(conv2d(lrelu(e2), gf_dim*4, name='g_e3_conv'))
        # e3 is (32 x 32 x gf_dim*4)
        e4 = g_bn_e4(conv2d(lrelu(e3), gf_dim*8, name='g_e4_conv'))
        # e4 is (16 x 16 x gf_dim*8)
        e5 = g_bn_e5(conv2d(lrelu(e4), gf_dim*8, name='g_e5_conv'))
        # e5 is (8 x 8 x gf_dim*8)
        e6 = g_bn_e6(conv2d(lrelu(e5), gf_dim*8, name='g_e6_conv'))
        # e6 is (4 x 4 x gf_dim*8)
        e7 = g_bn_e7(conv2d(lrelu(e6), gf_dim*8, name='g_e7_conv'))
        # e7 is (2 x 2 x gf_dim*8)
        e8 = g_bn_e8(conv2d(lrelu(e7), gf_dim*8, name='g_e8_conv'))
        # e8 is (1 x 1 x gf_dim*8)
        
        batch_size = params['batch_size']
        d1, d1_w, d1_b = deconv2d(tf.nn.relu(e8),
            [batch_size, s128, s128, gf_dim*8], name='g_d1', with_w=True)
        d1 = tf.nn.dropout(g_bn_d1(d1), 0.5)
        d1 = tf.concat([d1, e7], 3)
        # d1 is (2 x 2 x gf_dim*8*2)

        d2, d2_w, d2_b = deconv2d(tf.nn.relu(d1),
            [batch_size, s64, s64, gf_dim*8], name='g_d2', with_w=True)
        d2 = tf.nn.dropout(g_bn_d2(d2), 0.5)
        d2 = tf.concat([d2, e6], 3)
        # d2 is (4 x 4 x gf_dim*8*2)

        d3, d3_w, d3_b = deconv2d(tf.nn.relu(d2),
            [batch_size, s32, s32, gf_dim*8], name='g_d3', with_w=True)
        d3 = tf.nn.dropout(g_bn_d3(d3), 0.5)
        d3 = tf.concat([d3, e5], 3)
        # d3 is (8 x 8 x gf_dim*8*2)

        d4, d4_w, d4_b = deconv2d(tf.nn.relu(d3),
            [batch_size, s16, s16, gf_dim*8], name='g_d4', with_w=True)
        d4 = g_bn_d4(d4)
        d4 = tf.concat([d4, e4], 3)
        # d4 is (16 x 16 x gf_dim*8*2)

        d5, d5_w, d5_b = deconv2d(tf.nn.relu(d4),
            [batch_size, s8, s8, gf_dim*4], name='g_d5', with_w=True)
        d5 = g_bn_d5(d5)
        d5 = tf.concat([d5, e3], 3)
        # d5 is (32 x 32 x gf_dim*4*2)

        d6, d6_w, sd6_b = deconv2d(tf.nn.relu(d5),
            [batch_size, s4, s4, gf_dim*2], name='g_d6', with_w=True)
        d6 = g_bn_d6(d6)
        d6 = tf.concat([d6, e2], 3)
        # d6 is (64 x 64 x gf_dim*2*2)

        d7, d7_w, d7_b = deconv2d(tf.nn.relu(d6),
            [batch_size, s2, s2, gf_dim], name='g_d7', with_w=True)
        d7 = g_bn_d7(d7)
        d7 = tf.concat([d7, e1], 3)
        # d7 is (128 x 128 x gf_dim*1*2)

        d8, d8_w, d8_b = deconv2d(tf.nn.relu(d7),
            [batch_size, s, s, output_c_dim], name='g_d8', with_w=True)
        # d8 is (256 x 256 x output_c_dim)

        #return tf.nn.tanh(d8[:,:,:,:3]), tf.nn.tanh(d8[:,:,:,3:4])  #(w/o text , bin text)
        return tf.nn.tanh(d8)

In [ ]:
def discriminator(image, y=None, reuse=False):
    
    df_dim = params['df_dim']
    batch_size = params['batch_size']
    with tf.variable_scope("discriminator") as scope:

        # image is 256 x 256 x (input_c_dim + output_c_dim)
        if reuse:
            tf.get_variable_scope().reuse_variables()
        else:
            assert tf.get_variable_scope().reuse == False

        h0 = lrelu(conv2d(image, df_dim, name='d_h0_conv'))
        # h0 is (128 x 128 x df_dim)
        h1 = lrelu(d_bn1(conv2d(h0, df_dim*2, name='d_h1_conv')))
        # h1 is (64 x 64 x df_dim*2)
        h2 = lrelu(d_bn2(conv2d(h1, df_dim*4, name='d_h2_conv')))
        # h2 is (32x 32 x df_dim*4)
        h3 = lrelu(d_bn3(conv2d(h2, df_dim*8, d_h=1, d_w=1, name='d_h3_conv')))
        # h3 is (16 x 16 x df_dim*8)
        h4 = linear(tf.reshape(h3, [batch_size, -1]), 1, 'd_h3_lin')

        return tf.nn.sigmoid(h4), h4

In [ ]:
def GAN_model():
    
    real_A = tf.placeholder(dtype=tf.float32, shape=[params['batch_size'],256,256,1], name = 'real_A') #Arial,real
    real_B = tf.placeholder(dtype=tf.float32, shape=[params['batch_size'],256,256,1], name = 'real_B') #Times,real
    
    fake_B = generator(real_A) #Times,fake
    
    real_B_sum = tf.summary.image("Real_Times", real_B, max_outputs=1) 
    fake_B_sum = tf.summary.image("Fake_times", fake_B, max_outputs=1) 
    real_A_sum = tf.summary.image("Arial", real_A, max_outputs=1)
    tf.summary.histogram("Value_Range", fake_B)
    
    
    real_AB = tf.concat([real_A,real_B],3)
    fake_AB = tf.concat([real_A,fake_B],3)
    
    D_real,D_real_logits = discriminator(real_AB, reuse=False)
    D_fake,D_fake_logits = discriminator(fake_AB, reuse=True)
    
    d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real_logits, labels=tf.ones_like(D_real)))
    d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.zeros_like(D_fake)))
    d_loss = d_loss_real + d_loss_fake
    g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.ones_like(D_fake))) + params['L1_lambda'] * tf.reduce_mean(tf.abs(real_B - fake_B))
    
    g_loss_sum = tf.summary.scalar("g_loss", g_loss)
    d_loss_sum = tf.summary.scalar("d_loss", d_loss)
    
    t_vars = tf.trainable_variables()

    d_vars = [var for var in t_vars if 'd_' in var.name]
    g_vars = [var for var in t_vars if 'g_' in var.name]
    
    
    d_optim = tf.train.AdamOptimizer(0.0005).minimize(d_loss, var_list = d_vars)
    g_optim = tf.train.AdamOptimizer(0.0005).minimize(g_loss, var_list = g_vars)
    
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    summary = tf.summary.merge_all()
    trainwriter = tf.summary.FileWriter("./logs3/train", sess.graph)
    testwriter = tf.summary.FileWriter("./logs3/test", sess.graph)
    
    
    counter = 1
    start_time = time.time()
    
    trainpaths, testpaths = get_file_paths(params['path'])
    
    data_set_size = len(trainpaths)
    
    
    #sess = tf_debug.LocalCLIDebugWrapperSession(sess)
    print (data_set_size//params['batch_size'])
    for epoch in range(params['epochs']):
        
        for idx in range(data_set_size//params['batch_size']):
            
            batch_path = trainpaths[idx * params['batch_size'] : (idx + 1) * params['batch_size']]
            batch_data = np.array([load_data(path) for path in batch_path])
            print(batch_data.shape)
            KK = batch_data[:,:256]
            print(KK.shape)

            feed_dict = {real_A : batch_data[:,:256] , real_B : batch_data[:,256:]}
            
            _, d_loss_ = sess.run([d_optim, d_loss], feed_dict) # update D network
            _, g_loss_ = sess.run([g_optim, g_loss], feed_dict) # update G network
            _,summary_, g_loss_ = sess.run([g_optim,summary, g_loss], feed_dict) # update G network
            
            trainwriter.add_summary(summary_, counter)
            
            print ('#epoch : ' +str(epoch) + ' idx : ' +str(idx) + ' Dis loss : '+str(d_loss_) + ' Gen loss : '+str(g_loss_))
            
            if idx%10==0:
                
                batch_path_test = np.array(testpaths)[np.random.choice(len(testpaths),32)]
                batch_data_test = np.array([load_data(path) for path in batch_path_test])
                feed_dict_test = {real_A : batch_data_test[:,:256] , real_B : batch_data_test[:,256:]}
                test_summary_ = sess.run(summary, feed_dict_test)
                testwriter.add_summary(test_summary_,counter)
                _
            counter = counter + 1

In [2]:
171*5000

855000

In [ ]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

global sess

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

sess = tf.Session()
graph = tf.get_default_graph()


In [ ]:
GAN_model()